**Installing Packages**

In [ ]:
#@title
!pip install transformers
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.1 MB/s eta 0:00:00
--2023-05-14 03:45:01--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  13.4MB/s    in 7.3s    

2023-05-14 03:45:10 (11.1 MB/s) - ‘data/training.1600000.processed.noemoti

In [ ]:
#@title
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-05-14 03:45:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-14 03:45:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-14 03:45:36--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
#@title
import gensim.downloader as api
# load google news word2vec
w2v = api.load('word2vec-google-news-300')

[=================================================-] 99.9% 1661.7/1662.8MB downloaded


**Importing Libraries**

In [ ]:
import os
import csv
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from transformers import AdamW
from sklearn.model_selection import train_test_split
import torch
import datasets
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

**Loading The Data**

In [ ]:
import pandas as pd

df = pd.read_csv("data/training.1600000.processed.noemoticon.csv",
                names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                encoding='latin-1')
df.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.shape

(1600000, 6)

In [ ]:
df = df.drop(['id','date','query','user'],axis=1)
df

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['polarity'] = df['polarity'].replace(4,1)

In [ ]:
df['polarity'].value_counts()

0    800000
1    800000
Name: polarity, dtype: int64

**Get 0.2 of Data for Process**

In [ ]:
df_positive = df[df['polarity'] == 1].head(160000)
df_negetive = df[df['polarity'] == 0].head(160000)
df = pd.concat([df_positive,df_negetive])
print(df['polarity'].value_counts())
df

1    160000
0    160000
Name: polarity, dtype: int64


,polarity,text
800000,1,I LOVE @Health4UandPets u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...
800002,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,1,Being sick can be really cheap when it hurts t...
800004,1,@LovesBrooklyn2 he has that effect on everyone
...,...,...
159995,0,Home bored. Thinking of what to do with life.
159996,0,@KimokoMasada I wished I could tell you to sto...
159997,0,in London at the mo. It's ridiculously hot and...
159998,0,Just one more day in San Jose and then we're o...


**Helper Functions**: (preprocess, get_embeddings, remove not frequent words, get vocab and input_ids, collate batch)

In [ ]:
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re

nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
numbers = "[0-9]+"

def preprocess(text, stem=False):
  text = re.sub(
      r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?",
      "URL",
      text,
  )

  # replace mention
  text = re.sub(r"@[\w]+", "MENTION", text)

  # replace hashtag
  text = re.sub(r"#[\w]+", "HASHTAG", text)

  # replace dollar
  text = re.sub(r"\$\d+", "DOLLAR", text)

  # remove punctuation
  text = re.sub("[^a-zA-Z0-9]", " ", text)

  # remove multiple spaces
  text = re.sub(r"[\t\s\n]+", " ", text)
  text = re.sub(numbers, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

def collate_batch(batch):
  input_ids =[]
  labels = []
  for b in batch:
    input_ids.append(b["input_ids"])
    labels.append(b["polarity"])
  input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=0.0)
  return {"input_ids":input_ids ,"polarity":torch.Tensor(labels)}

def get_word_index(vocab):
  word2index = {}
  index2word = {}

  for i,w in enumerate(vocab):
    word2index[w] = i
    index2word[i] = w

  return word2index,index2word

# def list_remove(x,i):
#   for i in x:
#     if i in not_freq_words:
#       x.remove(i)
#   return x

def count_words(df,vocab):
  new_df = df
  new_df2 = df
  new_vocab = []
  count = {}
  for w in vocab:
    count[w] = 0
  new_df['token'].apply(lambda x: new_vocab.append(x))
  new_vocab = [item for tokens in new_vocab for item in tokens]
  for w in new_vocab:
    count[w] = count[w] + 1

  for k,v in count.items():
    if v <= 5:
      # print(new_df2['token'])
      new_df2['token'].apply(lambda x: x.remove(k) if k in x else x)

  return new_df2

def get_input_ids(token_list):
  id_list = []
  for t in token_list:
    id_list.append(word2index[t])
  return id_list

def get_vocab(df):
  vocab = []
  df['token'].apply(lambda x: vocab.append(x))
  vocab = [item for tokens in vocab for item in tokens]
  vocab = list(set(vocab))
  print('vocab_len:', len(vocab),'\n', 'vocab_head:',vocab[:20])
  return vocab

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.text = df.text.apply(lambda x: preprocess(x))
df

,polarity,text
800000,1,love mention u guys r best
800001,1,im meeting one besties tonight cant wait girl ...
800002,1,mention thanks twitter add sunisa got meet hin...
800003,1,sick really cheap hurts much eat real food plu...
800004,1,mention effect everyone
...,...,...
159995,0,home bored thinking life
159996,0,mention wished could tell stop tweeting cause ...
159997,0,london mo ridiculously hot left shorts home wa...
159998,0,one day san jose officially vacation get subbi...


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

df['token'] = df['text'].apply(word_tokenize)

df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,polarity,text,token
800000,1,love mention u guys r best,"[love, mention, u, guys, r, best]"
800001,1,im meeting one besties tonight cant wait girl ...,"[im, meeting, one, besties, tonight, cant, wai..."
800002,1,mention thanks twitter add sunisa got meet hin...,"[mention, thanks, twitter, add, sunisa, got, m..."
800003,1,sick really cheap hurts much eat real food plu...,"[sick, really, cheap, hurts, much, eat, real, ..."
800004,1,mention effect everyone,"[mention, effect, everyone]"
...,...,...,...
159995,0,home bored thinking life,"[home, bored, thinking, life]"
159996,0,mention wished could tell stop tweeting cause ...,"[mention, wished, could, tell, stop, tweeting,..."
159997,0,london mo ridiculously hot left shorts home wa...,"[london, mo, ridiculously, hot, left, shorts, ..."
159998,0,one day san jose officially vacation get subbi...,"[one, day, san, jose, officially, vacation, ge..."


In [ ]:
df2_positive = df[df['polarity'] == 1].head(8000)
df2_negetive = df[df['polarity'] == 0].head(8000)
df2 = pd.concat([df2_positive,df2_negetive])
vocab2 = get_vocab(df2)
print(len(vocab2))
new_df = count_words(df2,vocab2)
new_vocab = get_vocab(new_df)
print(len(new_vocab))

vocab_len: 12303 
 vocab_head: ['temps', 'hitler', 'easily', 'bedroom', 'listens', 'gert', 'odbc', 'encourage', 'rinn', 'cutest', 'nw', 'mockingbird', 'mexican', 'depressed', 'comics', 'rush', 'bedtime', 'bitches', 'crap', 'sushi']
12303
vocab_len: 2871 
 vocab_head: ['stupid', 'mess', 'bet', 'light', 'alone', 'easily', 'stage', 'bedroom', 'feelin', 'buffy', 'week', 'diego', 'hannah', 'ways', 'revisions', 'actually', 'morn', 'ruined', 'center', 'package']
2871


In [ ]:
word2index,index2word = get_word_index(vocab2)
def get_embedding_matrix_onehot():
  embedding_matrix = np.zeros((len(vocab2),len(vocab2)))

  for w in vocab2:
    embedding_matrix[word2index[w]][word2index[w]] = 1

  return embedding_matrix

In [ ]:
new_df['input_ids'] = new_df['token'].apply(get_input_ids)

In [ ]:
new_df = new_df[['input_ids','polarity']]

ds = Dataset.from_pandas(new_df)
ds = ds.train_test_split(test_size=0.2)
ds_train_valid = ds["train"].train_test_split(test_size=0.15)
train_dataset1 = ds_train_valid["train"]
validation_dataset = ds_train_valid["test"]
test_dataset1 = ds["test"]


train_dataset1.set_format(type='torch')
test_dataset1.set_format(type='torch')


In [ ]:
def validation( model , val_loader , loss_fn , device = "cuda"  ):
  val_loss = 0.0
  model.eval()
  reals = []
  preds_list = []
  for batch in val_loader:
    input_ids = batch["input_ids"].to(device)
    labels = batch["polarity"].to(device)
    with torch.no_grad():
      preds = model(input_ids = input_ids )
    loss = loss_fn(preds , labels.to(torch.long))
    val_loss += loss.item()
    reals += labels.cpu().numpy().tolist()
    preds_list += preds.argmax(axis=1).cpu().numpy().tolist()

  val_loss = val_loss / len(val_loader)
  accuracy = accuracy_score(reals, preds_list, normalize=True)
  print(classification_report(reals , preds_list ))
  return val_loss ,accuracy

In [ ]:
def train(model , optimizer , loss_fn , train_loader  , test_loader , epochs , device = "cuda"):

  model.to(device)
  for epoch in range(epochs):
    training_loss = 0.0
    model.train()
    t_labels = []
    t_preds = []

    for batch in tqdm_notebook(train_loader):
      optimizer.zero_grad()
      input_ids = batch["input_ids"].to(device)
      labels = batch["polarity"].to(device)
      preds = model(input_ids = input_ids)
      loss = loss_fn(preds , labels.to(torch.long))
      t_labels += labels.cpu().numpy().tolist()
      loss.backward()
      optimizer.step()
      training_loss += loss.item()
      t_preds +=  preds.argmax(axis=1).cpu().numpy().tolist()
    training_loss = training_loss / len(train_loader)
    train_accuracy =  accuracy_score(t_labels, t_preds, normalize=True)
    print('Epoch: {} ,training loss: {:.2f} , train accuracy: {:.2f} '.format(epoch , training_loss ,train_accuracy))
  test_loss , test_accuracy = validation(model , test_loader , loss_fn , device  )
  print('test loss: {:.2f}  test accuracy: {:.2f}'.format( test_loss , test_accuracy))

**One Hot - RNN**

In [ ]:
class RNN_module_onehot(nn.Module):
  def __init__(self,embedding_matrix):
    super(RNN_module_onehot,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix).float())
    self.fc1 = nn.Linear(len(vocab2), 150)
    self.rnn = nn.RNN(150,150,dropout=0.1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(150, 2)

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    out = self.fc1(embeddings)
    out,final_hidden = self.rnn(out)
    out = self.fc(final_hidden[0, :, :])
    return out

**One Hot - LSTM**

In [ ]:
class LSTM_module_onehot(nn.Module):
  def __init__(self,embedding_matrix):
    super(LSTM_module_onehot,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix).float())
    self.fc1 = nn.Linear(len(vocab2), 150)
    self.lstm = nn.LSTM(150,150,dropout=0.1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(150, 2)

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    out = self.fc1(embeddings)
    out,(final_hidden,cell_state) = self.lstm(out)
    out = self.fc(final_hidden[0, :, :])
    return out

**One Hot - GRU**

In [ ]:
class GRU_module_onehot(nn.Module):
  def __init__(self,embedding_matrix):
    super(GRU_module_onehot,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix).float())
    self.fc1 = nn.Linear(len(vocab2), 150)
    self.gru = nn.GRU(150,150,dropout=0.1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(150, 2)

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    out = self.fc1(embeddings)
    out,final_hidden = self.gru(out)
    out = self.fc(final_hidden[0, :, :])
    return out

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_onehot()
model = RNN_module_onehot(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset1 , batch_size=16, collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset1 , batch_size=16, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.77 , train accuracy: 0.48 


  0%|          | 0/680 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.76 , train accuracy: 0.49 


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.77 , train accuracy: 0.49 


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.76 , train accuracy: 0.49 
              precision    recall  f1-score   support

         0.0       0.50      0.96      0.65      1584
         1.0       0.53      0.05      0.09      1616

    accuracy                           0.50      3200
   macro avg       0.51      0.50      0.37      3200
weighted avg       0.51      0.50      0.37      3200

test loss: 0.71  test accuracy: 0.50


In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_onehot()
model = LSTM_module_onehot(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset1 , batch_size=16, collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset1 , batch_size=16, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.70 , train accuracy: 0.50 


  0%|          | 0/680 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.69 , train accuracy: 0.53 


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.69 , train accuracy: 0.53 


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.68 , train accuracy: 0.53 
              precision    recall  f1-score   support

         0.0       0.49      0.96      0.65      1584
         1.0       0.47      0.03      0.06      1616

    accuracy                           0.49      3200
   macro avg       0.48      0.50      0.36      3200
weighted avg       0.48      0.49      0.36      3200

test loss: 0.70  test accuracy: 0.49


In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_onehot()
model = GRU_module_onehot(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset1 , batch_size=16, collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset1 , batch_size=16, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.65 , train accuracy: 0.62 


  0%|          | 0/680 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.51 , train accuracy: 0.76 


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.48 , train accuracy: 0.78 


  0%|          | 0/680 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.48 , train accuracy: 0.78 
              precision    recall  f1-score   support

         0.0       0.72      0.66      0.69      1584
         1.0       0.69      0.74      0.72      1616

    accuracy                           0.70      3200
   macro avg       0.70      0.70      0.70      3200
weighted avg       0.70      0.70      0.70      3200

test loss: 0.66  test accuracy: 0.70


In [ ]:
vocab = get_vocab(df)

vocab_len: 95339 
 vocab_head: ['kajagoogoo', 'junkyard', 'officialshewmention', 'unauthenticated', 'nerdd', 'weeping', 'mony', 'impressing', 'playshop', 'rehoboth', 'lvn', 'headcolds', 'sheeesh', 'traced', 'suhweeeet', 'chile', 'twitterrr', 'pernah', 'carlisle', 'foriegn']


In [ ]:
def glove_embeddings():
  embedding_index = {}
  with open('glove.6B.300d.txt',encoding='utf-8') as f:
    for line in f:
      values = line.split()
      word = values[0]
      coef = np.asarray(values[1:],dtype="float32")
      embedding_index[word] = coef
  return embedding_index

In [ ]:
word2index,index2word = get_word_index(vocab)

In [ ]:
WORD_EMBED_DIM = 300
def get_embedding_matrix_glove():
  embedding_matrix = np.zeros((len(vocab),WORD_EMBED_DIM))
  unknown = np.zeros(WORD_EMBED_DIM)
  embeddings = glove_embeddings()
  word2index,index2word = get_word_index(vocab)

  for w in vocab:
    key = embeddings.get(w)
    if key is not None:
      embedding_matrix[word2index[w]] = embeddings[w]
    else:
      embedding_matrix[word2index[w]] = unknown

  average = np.mean(embedding_matrix)

  for w in vocab:
    key = embeddings.get(w)
    if key is None:
      embedding_matrix[word2index[w]] = average
  return embedding_matrix

In [ ]:
WORD_EMBED_DIM = 300

def get_embedding_matrix_w2v(w2v):
  embedding_matrix = np.zeros((len(vocab),WORD_EMBED_DIM))
  unknown = np.zeros(WORD_EMBED_DIM)

  for w in vocab:
    if w2v.has_index_for(w):
      embedding_matrix[word2index[w]] = w2v[w]
    else:
      embedding_matrix[word2index[w]] = unknown

  average = np.mean(embedding_matrix)

  for w in vocab:
    if not w2v.has_index_for(w):
      embedding_matrix[word2index[w]] = average
  return embedding_matrix

In [ ]:
df['input_ids'] = df['token'].apply(get_input_ids)
df

,polarity,text,token,input_ids
800000,1,love mention u guys r best,"[love, mention, u, guys, r, best]","[64105, 86768, 69128, 47007, 73206, 79316]"
800001,1,im meeting one besties tonight cant wait girl ...,"[im, meeting, one, tonight, cant, wait, girl, ...","[21698, 58536, 6781, 86127, 29182, 38288, 3679..."
800002,1,mention thanks twitter add sunisa got meet hin...,"[mention, thanks, twitter, add, got, meet, sho...","[86768, 7517, 16317, 15941, 37180, 28414, 1497..."
800003,1,sick really cheap hurts much eat real food plu...,"[sick, really, cheap, hurts, much, eat, real, ...","[69118, 72509, 40626, 65770, 24893, 60880, 432..."
800004,1,mention effect everyone,"[mention, effect, everyone]","[86768, 61800, 5372]"
...,...,...,...,...
159995,0,home bored thinking life,"[home, bored, thinking, life]","[49741, 67078, 41023, 16505]"
159996,0,mention wished could tell stop tweeting cause ...,"[mention, wished, could, tell, stop, tweeting,...","[86768, 73508, 88828, 33963, 77159, 18420, 702..."
159997,0,london mo ridiculously hot left shorts home wa...,"[london, mo, ridiculously, hot, left, shorts, ...","[15147, 58038, 62910, 73068, 12558, 52495, 497..."
159998,0,one day san jose officially vacation get subbi...,"[one, day, san, jose, officially, vacation, ge...","[6781, 66958, 50633, 52114, 90696, 16433, 5814..."


In [ ]:
df = df[['input_ids','polarity']]

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2)
ds_train_valid = ds["train"].train_test_split(test_size=0.15)
train_dataset = ds_train_valid["train"]
validation_dataset = ds_train_valid["test"]
test_dataset = ds["test"]


train_dataset.set_format(type='torch')
test_dataset.set_format(type='torch')


**RNN Module**

In [ ]:
class RNN_module(nn.Module):
  def __init__(self,embedding_matrix):
    super(RNN_module,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix).float())
    self.rnn = nn.RNN(WORD_EMBED_DIM,WORD_EMBED_DIM,dropout=0.1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(WORD_EMBED_DIM, 2)

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    out,final_hidden = self.rnn(embeddings)
    out = self.fc(final_hidden[0, :, :])
    return out

**RNN - glove - train and test**

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_glove()
model = RNN_module(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=200 , collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=200, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.70 , train accuracy: 0.51 


  0%|          | 0/1088 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.69 , train accuracy: 0.54 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.69 , train accuracy: 0.55 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.69 , train accuracy: 0.55 
              precision    recall  f1-score   support

         0.0       0.59      0.40      0.48     32023
         1.0       0.54      0.72      0.62     31977

    accuracy                           0.56     64000
   macro avg       0.57      0.56      0.55     64000
weighted avg       0.57      0.56      0.55     64000

test loss: 0.69  test accuracy: 0.56


**RNN - w2v - train and test**

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_w2v(w2v)
model = RNN_module(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=200 , collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=200, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.56 , train accuracy: 0.71 


  0%|          | 0/1088 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.51 , train accuracy: 0.75 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.51 , train accuracy: 0.76 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.50 , train accuracy: 0.76 
              precision    recall  f1-score   support

         0.0       0.76      0.76      0.76     32023
         1.0       0.76      0.76      0.76     31977

    accuracy                           0.76     64000
   macro avg       0.76      0.76      0.76     64000
weighted avg       0.76      0.76      0.76     64000

test loss: 0.51  test accuracy: 0.76


**LSTM Module**

In [ ]:
class LSTM_module(nn.Module):
  def __init__(self,embedding_matrix):
    super(LSTM_module,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix).float())
    self.lstm = nn.LSTM(WORD_EMBED_DIM,150,dropout=0.1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(150, 2)

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    out,(final_hidden,cell_state) = self.lstm(embeddings)
    out = self.fc(final_hidden[0, :, :])
    return out

**LSTM - glove - train and test**

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_glove()
model = LSTM_module(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=200 , collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=200, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.57 , train accuracy: 0.67 


  0%|          | 0/1088 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.46 , train accuracy: 0.78 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.43 , train accuracy: 0.80 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.41 , train accuracy: 0.81 
              precision    recall  f1-score   support

         0.0       0.79      0.72      0.76     32023
         1.0       0.75      0.81      0.78     31977

    accuracy                           0.77     64000
   macro avg       0.77      0.77      0.77     64000
weighted avg       0.77      0.77      0.77     64000

test loss: 0.50  test accuracy: 0.77


**LSTM - w2v - train and test**

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_w2v(w2v)
model = LSTM_module(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=200 , collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=200, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.53 , train accuracy: 0.73 


  0%|          | 0/1088 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.47 , train accuracy: 0.78 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.46 , train accuracy: 0.78 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.45 , train accuracy: 0.79 
              precision    recall  f1-score   support

         0.0       0.80      0.74      0.77     32023
         1.0       0.76      0.81      0.78     31977

    accuracy                           0.78     64000
   macro avg       0.78      0.78      0.78     64000
weighted avg       0.78      0.78      0.78     64000

test loss: 0.47  test accuracy: 0.78


**GRU Module**

In [ ]:
class GRU_module(nn.Module):
  def __init__(self,embedding_matrix):
    super(GRU_module,self).__init__()
    self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix).float())
    self.gru = nn.GRU(WORD_EMBED_DIM,150,dropout=0.1,batch_first=True)
    self.dropout = nn.Dropout(p=0.2)
    self.fc = nn.Linear(150, 2)

  def forward(self,input_ids):
    embeddings = self.word_embeddings(input_ids)
    out,final_hidden = self.gru(embeddings)
    out = self.fc(final_hidden[0, :, :])
    return out

**GRU - glove - train and test**

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_glove()
model = GRU_module(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=200 , collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=200, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.52 , train accuracy: 0.73 


  0%|          | 0/1088 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.45 , train accuracy: 0.78 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.43 , train accuracy: 0.80 


  0%|          | 0/1088 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.41 , train accuracy: 0.81 
              precision    recall  f1-score   support

         0.0       0.79      0.73      0.76     32023
         1.0       0.75      0.81      0.78     31977

    accuracy                           0.77     64000
   macro avg       0.77      0.77      0.77     64000
weighted avg       0.77      0.77      0.77     64000

test loss: 0.50  test accuracy: 0.77


**GRU - w2v - train and test**

In [ ]:
epoch = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_matrix = get_embedding_matrix_w2v(w2v)
model = GRU_module(embedding_matrix)
optimizer = AdamW(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset , batch_size=100 , collate_fn=collate_batch,drop_last = True)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size=100, drop_last=True , collate_fn=collate_batch )

train(model , optimizer , loss_fn , train_loader  , test_loader , epoch , device );

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2176 [00:00<?, ?it/s]

Epoch: 0 ,training loss: 0.51 , train accuracy: 0.75 


  0%|          | 0/2176 [00:00<?, ?it/s]

...

Epoch: 2 ,training loss: 0.45 , train accuracy: 0.78 


  0%|          | 0/2176 [00:00<?, ?it/s]

Epoch: 3 ,training loss: 0.44 , train accuracy: 0.79 


  0%|          | 0/2176 [00:00<?, ?it/s]

Epoch: 4 ,training loss: 0.42 , train accuracy: 0.80 
              precision    recall  f1-score   support

         0.0       0.77      0.79      0.78     32023
         1.0       0.78      0.77      0.77     31977

    accuracy                           0.78     64000
   macro avg       0.78      0.78      0.78     64000
weighted avg       0.78      0.78      0.78     64000

test loss: 0.47  test accuracy: 0.78
